<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [1]:
### YOUR CODE STARTS HERE
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

--2019-10-07 19:57:44--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.03s   

2019-10-07 19:57:44 (638 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [0]:
import pandas as pd
import numpy as np
import matplotlib as plt
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel, ttest_1samp

In [0]:
names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

names_0=['handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

In [73]:
df = pd.read_csv('house-votes-84.data', header=None, names=names)
print(df.shape)
df.head()

(435, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
df = df.replace({'?':np.NaN, 'n':0, 'y':1})

In [0]:
df.describe()

In [75]:
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
rep = df[df.party == 'republican']
rep = rep.drop(columns='party')
dem = df[df.party == 'democrat']
dem = dem.drop(columns='party')

In [93]:
rep.describe()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
count,165.000000,148.000000,164.000000,165.000000,165.000000,166.000000,162.000000,157.000000,165.000000,165.000000,159.000000,155.000000,158.000000,161.000000,156.000000,146.000000
mean,0.187879,0.506757,0.134146,0.987879,0.951515,0.897590,0.240741,0.152866,0.115152,0.557576,0.132075,0.870968,0.860759,0.981366,0.089744,0.657534
std,0.391804,0.501652,0.341853,0.109760,0.215442,0.304104,0.428859,0.361010,0.320176,0.498186,0.339643,0.336322,0.347298,0.135649,0.286735,0.476168
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [95]:
dem.describe()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
count,258.000000,239.000000,260.000000,259.000000,255.000000,258.000000,259.000000,263.000000,248.000000,263.000000,255.000000,249.000000,252.000000,257.000000,251.000000,185.000000
mean,0.604651,0.502092,0.888462,0.054054,0.215686,0.476744,0.772201,0.828897,0.758065,0.471483,0.505882,0.144578,0.289683,0.350195,0.637450,0.935135
std,0.489876,0.501045,0.315405,0.226562,0.412106,0.500430,0.420224,0.377317,0.429121,0.500138,0.500949,0.352383,0.454518,0.477962,0.481697,0.246956
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
t,r,d = [],[],[]

for x in rep.columns:
  t.append(ttest_ind(rep[x], dem[x], nan_policy= 'omit'))
  r.append(ttest_1samp(rep[x], 0.5, nan_policy='omit'))
  d.append(ttest_1samp(dem[x], 0.5, nan_policy='omit'))

In [0]:
r_0 = []
r_1 = []
for x in r:
  r_0.append(x[0])
  r_1.append(x[1])
d_0 = []
d_1 = []
for x in d:
  d_0.append(x[0])
  d_1.append(x[1])
t_0 = []
t_1 = []
for x in t:
  t_0.append(x[0])
  t_1.append(x[1])

In [206]:
df_0 = pd.DataFrame(data=[r_0,d_0,r_1,d_1,rep.mean(), dem.mean(), t_0, t_1], columns= names_0, index=['Rep_Stat','Dem_Stat','Rep_P', 'Dem_P', 'Rep_Mean', 'Dem_Mean', 'R_vs_D_Stat', 'R_vs_D_P'])
df_0

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
Rep_Stat,-1.023283e+01,0.163858,-1.370533e+01,5.709644e+01,2.692052e+01,1.684490e+01,-7.694446e+00,-1.204834e+01,-1.543983e+01,1.484534,-1.365951e+01,1.373240e+01,1.305701e+01,4.502695e+01,-1.787054e+01,3.997525e+00
Dem_Stat,3.431373e+00,0.064550,1.985941e+01,-3.167705e+01,-1.101688e+01,-7.464460e-01,1.042457e+01,1.413619e+01,9.470522e+00,-0.924685,1.875115e-01,-1.591578e+01,-7.345558e+00,-5.024597e+00,4.520728e+00,2.396576e+01
Rep_P,2.572179e-19,0.870068,6.249974e-29,3.888466e-110,4.531757e-62,1.103044e-37,1.343054e-12,4.758689e-24,8.750571e-34,0.139587,1.515766e-28,1.575474e-28,7.552694e-27,8.920944e-93,1.750902e-39,1.015490e-04
Dem_P,6.996123e-04,0.948587,5.759315e-54,6.796886e-91,2.500754e-23,4.560803e-01,1.832690e-21,4.190313e-34,2.359028e-18,0.355981,8.514093e-01,8.900199e-40,2.865381e-12,9.478257e-07,9.519196e-06,1.751376e-58
Rep_Mean,1.878788e-01,0.506757,1.341463e-01,9.878788e-01,9.515152e-01,8.975904e-01,2.407407e-01,1.528662e-01,1.151515e-01,0.557576,1.320755e-01,8.709677e-01,8.607595e-01,9.813665e-01,8.974359e-02,6.575342e-01
Dem_Mean,6.046512e-01,0.502092,8.884615e-01,5.405405e-02,2.156863e-01,4.767442e-01,7.722008e-01,8.288973e-01,7.580645e-01,0.471483,5.058824e-01,1.445783e-01,2.896825e-01,3.501946e-01,6.374502e-01,9.351351e-01
R_vs_D_Stat,-9.205264e+00,0.088965,-2.321278e+01,4.936708e+01,2.113669e+01,9.737576e+00,-1.252619e+01,-1.805209e+01,-1.643750e+01,1.735912,-8.293604e+00,2.050069e+01,1.351064e+01,1.634209e+01,-1.285315e+01,-6.849455e+00
R_vs_D_P,1.613440e-18,0.929156,2.070340e-77,1.994262e-177,5.600520e-68,2.393672e-20,8.521033e-31,2.824718e-54,5.030793e-47,0.083302,1.575932e-15,1.883420e-64,1.227858e-34,9.952343e-47,5.997697e-32,3.652674e-11


In [0]:
more_r = []
more_d = []

for x in df_0.iteritems():
  if(x[1]['Rep_Stat'] < 0 and x[1]['Rep_P'] < 0.01 and x[1]['Dem_Stat'] > 0 and x[1]['Dem_P'] < 0.01):
    more_r.append(x[0])
  elif(x[1]['Rep_Stat'] > 0 and x[1]['Rep_P'] < 0.01 and x[1]['Dem_Stat'] < 0 and x[1]['Dem_P'] < 0.01):
    more_d.append(x[0])

###1) Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
Issue(s) found:


In [203]:
more_d

['physician-fee-freeze',
 'el-salvador-aid',
 'education',
 'right-to-sue',
 'crime']

###2) Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
Issue(s) found: right-to-sue


In [204]:
more_r

['handicapped-infants',
 'budget',
 'anti-satellite-ban',
 'aid-to-contras',
 'mx-missile',
 'duty-free']

###3) Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)
Issue found: water-project